<a href="https://colab.research.google.com/github/chillburg/sentiment/blob/master/sentiment_Dat_Proc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Keras neural net approach**
This approach is training a neural net on coco_val dataset and then uses it to find similarities in the sentences from the assignment

In [1]:
from time import time
import pandas as pd
import numpy as np
from numpy import array
from nltk.corpus import stopwords
import random

from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from keras.models import Sequential, Model, load_model
from keras.layers.core import Dense, Activation, Flatten
from keras.optimizers import Adam, RMSprop, Adadelta
from keras.utils import normalize
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, TensorBoard
from keras.layers import Dropout, Conv2D, MaxPooling2D, Input, Dense, Flatten, concatenate, GlobalMaxPool2D, Lambda
from keras.layers import GlobalAveragePooling2D, Reshape, add, LSTM, TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Lambda
import keras.backend as K


from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Step 1 - formatting coco_val data for supervised learning

In [2]:
#corpus = pd.read_csv("/content/drive/My Drive/DatProc Group Assignment/coco_val.txt", header=None)
df_tweet = pd.read_json("/content/drive/My Drive/BALTA/Sentiment/tweet_corpus_edited.json", orient="str", encoding="ISO-8859-13").loc[:,["text",'sentiment'] ]
print(df_tweet.shape)
df_tweet.head

(1176, 2)


<bound method NDFrame.head of                                                    text sentiment
0     Nu gan laiks gulÄt (@ Laines mÄ«tne) http://t...       NEU
1     Ja jau neko, nekas neizpalÄ«dz, tad jau jÄdod...       NEG
2     es esmu tik Å”ausmÄ«gi dusmÄ«ga uz to ka savus...       NEG
3     Å”ovakar cirks ar sotiem bija rÄcÄ«gs ;D:D di...       POZ
4                               Ä¼oti bÄdÄ«gi palika .       NEG
...                                                 ...       ...
1171                                Zemgale 1:1 Kurbads       NEU
1172  @Diaaana_B nopirki kko lÄ«dzÄ«gu Latvijas Lott...       NEU
1173  @Santariii Tu mazias kaktuss bandas vadoni, ar...       NEU
1174  Vai es jums esmu teikusi ka mani draugi ir paÅ...       POZ
1175                Man uzbrÅ«k. http://t.co/gffM06Y9so       NEU

[1176 rows x 2 columns]>

Removing stopwords and stemming the rest with Lancaster

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
# stop_words = stopwords.words('latvian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()

def filter_stop_words(train_sentences, stop_words):
    for i, sentence in enumerate(train_sentences):
        #a = enumerate(train_sentences)
        #print(list(a))
        #print('sentence is:', i, sentence)
        new_sent = [lancaster.stem(word) for word in sentence.split() if word not in stop_words]
        print(new_sent)
        train_sentences[i] = ' '.join(new_sent)
    return train_sentences

stop_words = set(stopwords.words("english"))

print(corpus.shape)
print('corpus.iloc[2:10] is:\n', corpus.iloc[:10])
print()
corpus_no_stops = filter_stop_words(corpus.iloc[:10, 0], stop_words)
print('corpus_no_stops.iloc[:10]:', corpus_no_stops)


Assigning groups to coco-val sentences

In [ ]:
corpus.loc[0,1] = 1
count = 0
group = 0
for i in range(len(corpus)):
    #print(i)
    if count < 5:
        corpus.loc[i,1] = group
        count += 1
    else:
        group += 1
        corpus.loc[i,1] = group
        count = 1
    
corpus.head(30)

Making two columns, and nine rows for each sentence. In the second column each sentence has five correct pairs and four incorrect random pairs from the remaining corpus (further the same sentences should be removed)

In [ ]:
#data = pd.DataFrame(np.zeros((100, 3)).astype(int))
data = pd.DataFrame(np.zeros((1, 3)).astype(int))

for i in range(len(corpus)):  # for each line in corpus
    a = pd.DataFrame()
    for j in range(9):            # 9 loops, 5 for True and 4 for False
        a.loc[0, 0] = corpus.loc[i, 0]             # places the corpus line
        if j < 5:                           # first five match, rest don't
            same_gr = corpus.loc[corpus[1] == corpus.loc[i, 1]]
            a.loc[0, 1] = same_gr.iloc[j, 0]       
        else:                                        # if not, a random line from other groups
            other = corpus.loc[corpus[1] != corpus.loc[i, 1]]
            other_text = other.iloc[random.randint(0, len(other)-5)]
            a.loc[0, 1] = other_text[0]      
        if j < 5:
            a.loc[0, 2] = True
        else:
            a.loc[0, 2] = False
        data = pd.concat([data,a])
data = data.iloc[1:, :]
data

Exporting the data as csv

In [ ]:
export_csv = data.to_csv('/content/drive/My Drive/DatProc Group Assignment/coco_T_F_225000_stemmed.csv', header=False)

### **Step 2 - dowloading and formatting the assignment data**

In [ ]:
with open('/content/drive/My Drive/DatProc Group Assignment/competition_descriptions.txt') as fp:
    corpus = fp.readlines()
    fp.close()

Removing special characters

In [ ]:
corpus_cl = corpus
for i in enumerate(corpus):
    corpus_cl[i[0]] = i[1].replace("[comma]", "").lower().replace(".", "").replace("\n", "").replace("'", " ").replace("-", " ").replace("/", " ").replace("?", " ").replace("(", " ").replace(")", " ").strip()     #'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

Removing stopwords and stemming

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()


def filter_stop_words(train_sentences, stop_words):
    for i, sentence in enumerate(train_sentences):
        new_sent = [lancaster.stem(word) for word in sentence.split() if word not in stop_words]
        train_sentences[i] = ' '.join(new_sent)
    return train_sentences

stop_words = set(stopwords.words("english"))

corpus_cl = filter_stop_words(corpus, stop_words)
print('corpus_cl:', corpus_cl[:10])


Filling a data frame with each sentence corresponding to every other sentence in the assignment data

In [ ]:
data = pd.DataFrame(np.zeros((1, 2)).astype(int))

for i in range(len(corpus_cl)):  # for each line in corpus
    print(i)
    a = pd.DataFrame()
    for j in range(len(corpus_cl)):
        a.loc[0, 0] = corpus_cl[i]             # places the corpus line
        a.loc[0, 1] = corpus_cl[j]
        data = pd.concat([data,a])
data = data.iloc[1:, :]
print(data.shape)
data

Saving the data as csv

In [ ]:
data.to_csv('test_set_595985.csv')

### **Step 3 - Training the model on coco_val grouped data**

Loading transformed coco_val

In [ ]:
a = pd.read_csv('/content/drive/My Drive/DatProc Group Assignment/coco_T_F_225000_stemmed.csv', dtype=str, header = None).iloc[:, 1:]
print(a.shape)
a.head()

(225000, 3)


,1,2,3
0,child hold flow umbrell pet yak,child hold flow umbrell pet yak,1
1,child hold flow umbrell pet yak,young man hold umbrell next herd cattl,1
2,child hold flow umbrell pet yak,young boy barefoot hold umbrell touch horn cow,1
3,child hold flow umbrell pet yak,young boy umbrell touch horn cow,1
4,child hold flow umbrell pet yak,boy hold umbrell stand next livestock,1


Concatenating both columns together, as this model is a single-input one

In [ ]:
a['space'] = ' '
a['both'] = a[[1, 'space', 2]].apply(lambda x: ''.join(x), axis=1)

print('a.shape:', a.shape)
print('a.iloc[0, 4]:', a.iloc[0, 4])
docs = a.iloc[:, 4]
print('docs.shape:', docs.shape)
print('docs.iloc[:3]:', docs.iloc[:3])
labels = a.iloc[:, 2]
print('labels.shape:', labels.shape)
print('labels.iloc[:10]:', labels.iloc[:10])

a.shape: (225000, 5)
a.iloc[0, 4]: child hold flow umbrell pet yak child hold flow umbrell pet yak
docs.shape: (225000,)
docs.iloc[:3]: 0    child hold flow umbrell pet yak child hold flo...
1    child hold flow umbrell pet yak young man hold...
2    child hold flow umbrell pet yak young boy bare...
Name: both, dtype: object
labels.shape: (225000,)
labels.iloc[:10]: 0    1
1    1
2    1
3    1
4    1
5    0
6    0
7    0
8    0
9    1
Name: 3, dtype: object


Training the model

In [ ]:
vocab_size = 150
print('vocabulary size is:', vocab_size)

encoded_docs = [one_hot(d, vocab_size) for d in docs]
print('encoded_docs:', encoded_docs[0])

# pad documents to a max length of x words
max_length = vocab_size

padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print('padded_docs:\n', padded_docs)

# define the model
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_length))
model.add(Flatten())

model.add(BatchNormalization())
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.3))

model.add(BatchNormalization())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.4))
    
model.add(BatchNormalization())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer = Adam(lr = 0.000005), loss='binary_crossentropy', metrics=['accuracy'])

# summarize the model
print(model.summary())

# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=1, validation_split = 0.2, shuffle = True, batch_size = 64)

# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

vocabulary size is: 150
encoded_docs: [16, 45, 138, 60, 45, 109, 16, 45, 138, 60, 45, 109]
padded_docs:
 [[ 16  45 138 ...   0   0   0]
 [ 16  45 138 ...   0   0   0]
 [ 16  45 138 ...   0   0   0]
 ...
 [  1 139 132 ...   0   0   0]
 [  1 139 132 ...   0   0   0]
 [  1 139 132 ...   0   0   0]]
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 128)          19200     
_________________________________________________________________
flatten_1 (Flatten)          (None, 19200)             0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 19200)             76800     
______________________________________________________

KeyboardInterrupt: ignored

### **Step 4 - predict similar sentences in assignment data with the model trainded on coco_val dataset**

Downloading test set

In [ ]:
a_test = pd.read_csv('/content/drive/My Drive/DatProc Group Assignment/test_set_595985.csv', dtype=str, header = None).iloc[1:, 1:]
print(a_test.shape)
print(a_test.head())

(595984, 2)
                                                   1                                                  2
1  beauty eyebrow nic hair cut complect cle young...  beauty eyebrow nic hair cut complect cle young...
2  beauty eyebrow nic hair cut complect cle young...  short dark hair light brown ol skin thick red ...
3  beauty eyebrow nic hair cut complect cle young...  as guy 20s short hair dark skin ton monolid ey...
4  beauty eyebrow nic hair cut complect cle young...  person crew cut dirty blond hair squ fac round...
5  beauty eyebrow nic hair cut complect cle young...  long hair comb emo lik beard look scraggily we...


Concatenating the test set

In [ ]:
a_test['space'] = ' '
a_test['both'] = a_test[[1, 'space', 2]].apply(lambda x: ''.join(x), axis=1)

print('a_test.shape:', a_test.shape)
print('a_test.iloc[0, 3]:', a_test.iloc[0, 3])
docs_test = a_test.iloc[1:, 3]
print('docs_test.shape:', docs_test.shape)
print('docs_test.iloc[:3]:', docs_test.iloc[:3])

a_test.shape: (595985, 4)
a_test.iloc[0, 3]: 0 1
docs_test.shape: (595984,)
docs_test.iloc[:3]: 1    beauty eyebrow nic hair cut complect cle young...
2    beauty eyebrow nic hair cut complect cle young...
3    beauty eyebrow nic hair cut complect cle young...
Name: both, dtype: object


Padding the test set

In [ ]:
vocab_size_test = 150
print('vocabulary size is:', vocab_size_test)

encoded_docs_test = [one_hot(d, vocab_size_test) for d in docs_test]
print('encoded_docs_test:', encoded_docs_test[0])

# pad documents to a max length of x words
max_length_test = vocab_size_test

padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length_test, padding='post')
print('padded_docs:\n', padded_docs_test)

vocabulary size is: 150
encoded_docs_test: [87, 118, 128, 116, 3, 8, 122, 94, 143, 125, 107, 48, 142, 95, 69, 11, 130, 134, 87, 118, 128, 116, 3, 8, 122, 94, 143, 125, 107, 48, 142, 95, 69, 11, 130, 134]
padded_docs:
 [[ 87 118 128 ...   0   0   0]
 [ 87 118 128 ...   0   0   0]
 [ 87 118 128 ...   0   0   0]
 ...
 [ 98  61  82 ...   0   0   0]
 [ 98  61  82 ...   0   0   0]
 [ 98  61  82 ...   0   0   0]]


Predicting pairs

In [ ]:
predictions = model.predict(padded_docs_test)
print(predictions.shape)
print(predictions[:2])

In [ ]:
np.savetxt('predictions_595985.txt', predictions)

In [ ]:
predictions = np.loadtxt('/content/predictions_595985.txt')

In [ ]:
a_united.iloc[9263, [0, 3]]



1           person black short black hair bit fac stubbl a...
question                                                   11
Name: 9264, dtype: object

In [ ]:
print(a_test.shape, predictions.shape)


a_united = a_test
a_united['predictions'] = predictions
a_united['question'] = 0

print('a_united.shape:', a_united.shape)
print(a_united.head(2))

#### crash
for i in range(772):
    a_united.iloc[0+i*772:772+i*772, 3] = i


(595984, 4) (595984,)
a_united.shape: (595984, 4)
                                                   1  ... question
1  beauty eyebrow nic hair cut complect cle young...  ...        0
2  beauty eyebrow nic hair cut complect cle young...  ...        0

[2 rows x 4 columns]


In [ ]:
grouped = a_united.groupby(2)['predictions']
a_united['second_lowest'] = grouped.transform(lambda x: x.nlargest(1).max())
print(a_united.head())

np.savetxt('a_united.txt', a_united)




def find_similar_preview(line_no):
    j = [line_no]
    j.append(Z[line_no, :].argsort()[-2])
    j.append(Z[line_no, :].argsort()[-3])
    j.append(Z[line_no, :].argsort()[-4])
    print("Most similar sentences to [", line_no, "] according to cosine similarity:\n", sep='')
    print(corpus[line_no])
    print('\nare numbers ', j)
    #print(corpus[j[0]])
    print(corpus[j[1]])
    print(corpus[j[2]])
    print(corpus[j[3]])
    


def find_similar(line_no):
    j = [line_no]
    j.append(Z[line_no, :].argsort()[-2])
    j.append(Z[line_no, :].argsort()[-3])
    j.append(Z[line_no, :].argsort()[-4])
    return j


                                                   1  ... second_lowest
1  beauty eyebrow nic hair cut complect cle young...  ...      1.000000
2  beauty eyebrow nic hair cut complect cle young...  ...      1.000000
3  beauty eyebrow nic hair cut complect cle young...  ...      1.000000
4  beauty eyebrow nic hair cut complect cle young...  ...      1.000000
5  beauty eyebrow nic hair cut complect cle young...  ...      0.999995

[5 rows x 5 columns]


In [ ]:
grouped = a_united.groupby(1)['predictions']
a_united['second_lowest'] = grouped.transform(lambda x: x.nlargest(4).max())

print(a_united.head(1))

                                                   1  ... second_lowest
1  beauty eyebrow nic hair cut complect cle young...  ...      0.999999

[1 rows x 5 columns]


In [ ]:
print(a_test.shape)    # (595984, 3)
print(a_test.iloc[:10, 0]) 
print('0-------------------------')
print(a_test.iloc[:10, 1]) 
print('1-------------------------')
print(a_test.iloc[:10, 2])
print('2-------------------------')

z = a_test.groupby(by = 1)[2].max()
print('-------------------------')
print(z.shape)
print(z.head(4))


(595984, 3)
1     beauty eyebrow nic hair cut complect cle young...
2     beauty eyebrow nic hair cut complect cle young...
3     beauty eyebrow nic hair cut complect cle young...
4     beauty eyebrow nic hair cut complect cle young...
5     beauty eyebrow nic hair cut complect cle young...
6     beauty eyebrow nic hair cut complect cle young...
7     beauty eyebrow nic hair cut complect cle young...
8     beauty eyebrow nic hair cut complect cle young...
9     beauty eyebrow nic hair cut complect cle young...
10    beauty eyebrow nic hair cut complect cle young...
Name: 1, dtype: object
0-------------------------
1     beauty eyebrow nic hair cut complect cle young...
2     short dark hair light brown ol skin thick red ...
3     as guy 20s short hair dark skin ton monolid ey...
4     person crew cut dirty blond hair squ fac round...
5     long hair comb emo lik beard look scraggily we...
6     slight long hair wear black shirt light new kn...
7           short stat thin build cas hair

In [ ]:
a_united.iloc[:773, 4]
predictions[:773].sort()


array([0.02608377, 0.04240781, 0.05693784, 0.05770829, 0.05775827,
       0.05898771, 0.06319138, 0.06597468, 0.07404107, 0.07722339,
       0.08301091, 0.0850133 , 0.08826339, 0.08899859, 0.08943892,
       0.09699497, 0.10174325, 0.10423276, 0.10515809, 0.10869712,
       0.11127746, 0.11491409, 0.11844182, 0.12101236, 0.12570336,
       0.12788451, 0.13190243, 0.13343292, 0.13867936, 0.1413084 ,
       0.14363325, 0.14365393, 0.144151  , 0.14897257, 0.15035373,
       0.1514931 , 0.15162063, 0.15307364, 0.15992811, 0.16416013,
       0.16440064, 0.17778501, 0.17902544, 0.18065643, 0.18445095,
       0.18635496, 0.18994078, 0.19239122, 0.20057392, 0.20148352,
       0.2022633 , 0.2054989 , 0.2115517 , 0.21628767, 0.21842697,
       0.21925154, 0.22118935, 0.2250571 , 0.22708312, 0.22718021,
       0.22764957, 0.22851342, 0.23435557, 0.23531231, 0.2392976 ,
       0.23984358, 0.24142399, 0.24328056, 0.24863347, 0.25113654,
       0.25322774, 0.25746942, 0.25750586, 0.26109174, 0.26401